# What is this

A place to store old alternative code for things just in case we want to go back and look at it. Copy and paste in "Replication notebook" to see how it works. 

In [1]:
import numpy as np
import pandas as pd
import geopandas as geo

import cvxpy as cp
import numpy.linalg as LA
import statsmodels.api as sm
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import statsmodels.formula.api as smf
from joblib import Parallel, delayed
from scipy.optimize import differential_evolution, NonlinearConstraint, Bounds

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [5.5, 3.5]
#plt.rcParams['figure.figsize'] = [6, 4.0]
#plt.rcParams['figure.dpi'] = 80

In [2]:
dtafile = './dataset/Pinotti-replication/dataset.dta'

### Alternative way to illustrate Fig 2.1 & Fig 2.3
Author: Danial. <br>
Removed by: Jessica. <br>
Value: alternative `plt.annotate()` method.

In [ ]:
#######Alternative way to illustrate Fig 2.1 & Fig 2.3######
# J: I'd keep the previous way

####Fig 2.1####
df2.plot.scatter('mafia', 'gdppercap', c=color, s=10, linewidth=3,
                xlabel='Presence of Criminal Organisations', ylabel='GDP per capita',
                title='Figure 1: mafia-type crimial organizations and GDP per capita across Italian regions, average over the period 1983-2007')
plt.rcParams["figure.figsize"] = (8,5)

for (i, (x,y)) in enumerate(zip(df2['mafia'],df2['gdppercap'])):
    if df2['region'][i] in ['SIC','BAS','PUG','CAM','CAL']:
         plt.annotate(df2['region'][i],(x,y))  

####Fig 2.3####
df2.plot.scatter('mafia', 'murd', c=color, s=10, linewidth=3,
                xlabel='Homicides x 100,000 Inhabitants', ylabel='Mafia Allegations (art. 416 bis) x 100,000 Inhabitants',
                title='Figure 3: presence over time of mafia-type criminal organizations in different areas in Italy, years 1983-2007')
plt.rcParams["figure.figsize"] = (8,5)

for (i, (x,y)) in enumerate(zip(df2['mafia'],df2['murd'])):
    if df2['region'][i] in ['SIC','BAS','PUG','CAM','CAL']:
         plt.annotate(df2['region'][i],(x,y))  


# Section 3

In [14]:
# dtafile already defined, unnecessary
dtafile = './dataset/Pinotti-replication/dataset.dta'

data = pd.read_stata(dtafile)

# Specify conditions for treated unit and control units as per Pinotti's paper (c.f. F216), 
# 21 is "NEW" Recent mafia presence: Apulia and Basilicata

treat_unit     = data[data.reg == 21]
treat_unit     = treat_unit[treat_unit.year <= 1960]                 # Matching period: 1951 to 1960
treat_unit_all = data[data.reg == 21]                                # Entire period:   1951 to 2007

control_units     = data[(data.reg <= 14) | (data.reg ==20)]
control_units     = control_units[control_units.year <= 1960]
control_units_all = data[(data.reg <= 14) | (data.reg ==20)]

# Extract the outcome variable for treatment and control unit, y: GDP per capita

y_treat     = np.array(treat_unit.gdppercap).reshape(1, 10)              # Matching period: 1951 to 1960
y_treat_all = np.array(treat_unit_all.gdppercap).reshape(1, 57)          # Entire period:   1951 to 2007

y_control     = np.array(control_units.gdppercap).reshape(15, 10)
y_control_all = np.array(control_units_all.gdppercap).reshape(15, 57)

Z1 = y_treat.T      # Transpose
Z0 = y_control.T

## Prepare matrices with only the relevant variables into CVXPY format, predictors k = 8
predictor_variables = ['gdppercap', 'invrate', 'shvain', 'shvaag', 'shvams', 'shvanms', 'shskill', 'density']
X = data.loc[data['year'].isin(list(range(1951, 1961)))]
X.index = X.loc[:,'reg']

#####################################################################################
##### WHY DO X0 AND X1 END UP WITH SUPER SMALL NUMBERS IF THEY ARE .mean() ??   #####
#####################################################################################


# k x 1 vector: mean values of k predictors for 1 treated unit
X0 = X.loc[(X.index <= 14) | (X.index ==20),(predictor_variables)] 
X0 = X0.groupby(X0.index).mean().values.T

# k x J matrix: mean values of k predictors for J untreated units
X1 = X.loc[(X.index == 21),(predictor_variables)]
X1 = X1.groupby(X1.index).mean().values.T

In [17]:
#X
#X0
X1

array([[2.3949956e+03],
       [3.1629604e-01],
       [2.1547881e-01],
       [1.5170218e-01],
       [4.0166542e-01],
       [2.3115355e-01],
       [1.6548637e-01],
       [1.3477887e+02]], dtype=float32)

In [ ]:
# CVXPY Setup: Define function to call and output a vector of weights function

def w_optimize(v=None):                                                         ## v is the parameter of the function
    
    V = np.zeros(shape=(8, 8))
    
    if v is None:
        np.fill_diagonal(V, np.ones(8))
    
    else:                                                                        ## Is the else part necessary?
        np.fill_diagonal(V, v)

    W = cp.Variable((15, 1), nonneg=True)                                        ## Matrix variable with shape (15, 1), nonnegative elements
    objective_function    = cp.Minimize(cp.sum(V @ cp.square(X1 - X0 @ W)))      ## cp.Minimize(function to minimize) ; 
                                                                                 ## cp.sum(expression) sums the entries of an expression ; 
                                                                                 ## cp.square() is ^2
    objective_constraints = [cp.sum(W) == 1]
    objective_solution    = cp.Problem(objective_function, objective_constraints).solve(verbose=False)
                            ## cp.Problem(function to minimize, constraints)
                            ##   .solve() to gen solution
    
    return (W.value,objective_solution)

# CVXPY Solution
w_basic, objective_solution = w_optimize()             ## where does w_basic come from, what is it doing?
print('\nObjective Value: ',objective_solution,'\n\nOptimal Weights: ',w_basic.T)
solution_frame_1 = pd.DataFrame({'Region':control_units.region.unique(), 
                           'Weights': np.round(w_basic.T[0], decimals=3)})

display(solution_frame_1)

In [ ]:
#### Graphical Comparison

w_pinotti = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.624, 0.376, 0]).reshape(15, 1)
w_becker = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.43035, 0.48934, 0.0803045]).reshape(15,1)

y_synth_pinotti = w_pinotti.T @ y_control_all      ## generate the synthetic control output values by weight * y for control
y_synth_becker = w_becker.T @ y_control_all
y_synth_basic = w_basic.T @ y_control_all

fig = go.Figure()                    ## using plotly.graph_objs
fig.add_trace(go.Scatter(x=list(data.year.unique()), y=y_synth_basic[0],
                    mode='lines', name='Optimizer'))
fig.add_trace(go.Scatter(x=list(data.year.unique()), y=y_synth_pinotti[0],
                    mode='lines', name='Pinotti'))
fig.add_trace(go.Scatter(x=list(data.year.unique()), y=y_synth_becker[0],
                    mode='lines', name='Becker'))
fig.add_trace(go.Scatter(x=list(data.year.unique()), y=y_treat_all[0],
                    mode='lines', name='Treated unit'))

fig.add_shape(dict(type="line", x0=1960, y0=0, x1=1960, y1=11000,
                   line=dict(color="Black", width=1)))

fig.add_shape(dict(type="line", x0=1974, y0=0, x1=1974, y1=11000,
                   line=dict(color="Black", width=1)))

fig.add_shape(dict(type="line", x0=1980, y0=0, x1=1980, y1=11000,
                   line=dict(color="Black", width=1)))

fig.add_trace(go.Scatter(x=[1960], y=[12000], mode="text",
    name="Matching", text=["End of Matching<br>Period"]))

fig.add_trace(go.Scatter(x=[1974], y=[12000], mode="text",      ## why??
    name="Event 1", text=["Drug<br>Smuggling"]))

fig.add_trace(go.Scatter(x=[1981], y=[12000], mode="text",
    name="Event 2", text=["Basilicata<br>Earthquake"]))

fig.update_layout(title='Synthetic Control<br>Optimizer vs. Treated unit',
                   xaxis_title='Time', yaxis_title='GDP per Capita')

# Dynamic graph
fig.show()

# Static graph only for display on github
#fig.show(renderer="png")